In [1]:
import pandas as pd

In [2]:
bat_df = pd.read_csv('../Datasets/batter_stats.csv')
pitch_df = pd.read_csv('../Datasets/pitcher_stats.csv')
lineups = pd.read_csv('../Datasets/Lineups_2021_06_15.csv')

In [3]:
# The dots in names like J.T. were inconsistent across datasets
bat_df.loc[:,'Name'] = bat_df['Name'].str.replace('.', '')
pitch_df.loc[:,'Name'] = pitch_df['Name'].str.replace('.', '')
lineups.loc[:,' player name'] = lineups[' player name'].str.replace('.', '')

In [4]:
bats_r = bat_df[bat_df['bats'] == 'R']
bats_l = bat_df[bat_df['bats'] == 'L']

In [5]:
# Making name column consistent
lineups.columns = lineups.columns.str.replace(' player name', 'Name')

In [6]:
# Individual name that was giving trouble
lineups.loc[lineups['Name'] == 'Jonathan Gray', 'Name'] = 'Jon Gray'

In [7]:
# Creates a list of teams with no repeats
# The order needs to be kept from the dataframe because 
# opposing teams are paired
team_list = []
for team in lineups['team code']:
    if team not in team_list:
        team_list.append(team)
# Creating a dictionary of every team, and the team they face
matchups = {}
for i in range(1, len(team_list), 2):
    matchups[team_list[i]] = team_list[i-1]
    matchups[team_list[i-1]] = team_list[i]

In [8]:
lineups

,team code,game_date,game_number,mlb id,Name,batting order,confirmed,position
0,PIT,6/15/2021,1,624428,Adam Frazier,1,N,NaN
1,PIT,6/15/2021,1,663647,Ke'Bryan Hayes,2,N,NaN
2,PIT,6/15/2021,1,668804,Bryan Reynolds,3,N,NaN
3,PIT,6/15/2021,1,607732,Jacob Stallings,4,N,NaN
4,PIT,6/15/2021,1,595943,Phillip Evans,5,N,NaN
...,...,...,...,...,...,...,...,...
279,LAD,6/15/2021,1,621035,Chris Taylor,5,N,NaN
280,LAD,6/15/2021,1,666158,Gavin Lux,6,N,NaN
281,LAD,6/15/2021,1,656716,Zach McKinstry,7,N,NaN
282,LAD,6/15/2021,1,572041,AJ Pollock,8,N,NaN


In [9]:
batters = lineups.loc[(lineups[' batting order'] != 'SP') & (lineups[' game_number'] == 1)]

In [10]:
pitch_df.drop(456, inplace = True)

In [11]:
matchups

{'WAS': 'PIT',
 'PIT': 'WAS',
 'TOR': 'NYY',
 'NYY': 'TOR',
 'NYM': 'CHC',
 'CHC': 'NYM',
 'CLE': 'BAL',
 'BAL': 'CLE',
 'ATL': 'BOS',
 'BOS': 'ATL',
 'HOU': 'TEX',
 'TEX': 'HOU',
 'CHW': 'TB',
 'TB': 'CHW',
 'MIL': 'CIN',
 'CIN': 'MIL',
 'KC': 'DET',
 'DET': 'KC',
 'STL': 'MIA',
 'MIA': 'STL',
 'COL': 'SD',
 'SD': 'COL',
 'OAK': 'LAA',
 'LAA': 'OAK',
 'SF': 'ARI',
 'ARI': 'SF',
 'SEA': 'MIN',
 'MIN': 'SEA',
 'LAD': 'PHI',
 'PHI': 'LAD'}

In [12]:
pitch_df.loc[pitch_df['Name']== 'Chase DeJong', 'throws']

Series([], Name: throws, dtype: object)

In [13]:
opp_pitch_hand = []
opp_pitcher = []
del_indexes = []
keep_pitch = []
for i, row in batters.iterrows():
    opp_pitch = lineups.loc[(lineups['team code'] == matchups[row['team code']])\
                            & (lineups[' batting order'] == 'SP') & (lineups[' game_number'] == 1), 'Name'].values[0]
    if len(pitch_df.sort_values(by = 'IP', ascending = False).loc[pitch_df['Name'] == opp_pitch, 'throws'].values) > 0:
            opp_pitch_hand.append(pitch_df.sort_values(by = 'IP', ascending = False).loc[pitch_df['Name'] == opp_pitch, 'throws'].values[0])
            opp_pitcher.append(opp_pitch)
            keep_pitch.append(opp_pitch)
    else:
        del_indexes.append(i)
        
batters.drop(index = del_indexes, inplace = True)
batters['opp_pitch_hand'] = opp_pitch_hand
batters['opp_pitch'] = opp_pitcher


/Users/nperry725/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
<ipython-input-13-9eb6f881f0bf>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  batters['opp_pitch_hand'] = opp_pitch_hand
<ipython-input-13-9eb6f881f0bf>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.htm

In [14]:
del_indexes

[18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 228,
 229,
 230,
 231,
 232,
 233,
 234,
 235]

In [15]:
bats = []
k_pct = []
bb_pct = []
bb_k = []
bat_avg = []
obp = []
slg = []
ops = []
iso = []
babip = []
wrc = []
wraa = []
woba = []
wrc_plus = []
for i, row in batters.iterrows():
    # Pulling stats based on handedness of pitcher
    if row['opp_pitch_hand'] == 'R':
        batter_df = bat_df.loc[(bat_df['pitch']== 'R')&(bat_df['Name']==row['Name'])]
        # Checking for switch hitters
        if len(batter_df) == 2:
            # A switch hitter will hit using the opposite hand of the pitcher
            batter_df = batter_df.loc[batter_df['bats']== 'L']
    if row['opp_pitch_hand'] == 'L':
        batter_df = bat_df.loc[(bat_df['pitch']== 'L')&(bat_df['Name']==row['Name'])]
        # Checking for switch hitters
        if len(batter_df) == 2:
            # A switch hitter will hit using the opposite hand of the pitcher
            batter_df = batter_df.loc[batter_df['bats']== 'R']
    if len(batter_df) == 0:
        for col in batter_df.columns:
            batter_df[col] = [float('NaN')]
    bats.append(batter_df['bats'].values[0])
    k_pct.append(batter_df['K%'].values[0])
    bb_pct.append(batter_df['BB%'].values[0])
    bb_k.append(batter_df['BB/K'].values[0])
    bat_avg.append(batter_df['AVG'].values[0])
    obp.append(batter_df['OBP'].values[0])
    slg.append(batter_df['SLG'].values[0])
    ops.append(batter_df['OPS'].values[0])
    iso.append(batter_df['ISO'].values[0])
    babip.append(batter_df['BABIP'].values[0])
    wrc.append(batter_df['wRC'].values[0])
    wraa.append(batter_df['wRAA'].values[0])
    woba.append(batter_df['wOBA'].values[0])
    wrc_plus.append(batter_df['wRC+'].values[0])
batters.loc[:,'bats'] = bats
batters.loc[:,'k_pct'] = k_pct
batters.loc[:,'bb_pct'] = bb_pct
batters.loc[:,'bb_k'] = bb_k
batters.loc[:,'bat_avg'] = bat_avg
batters.loc[:,'obp'] = obp
batters.loc[:,'slg'] = slg
batters.loc[:,'ops'] = ops
batters.loc[:,'iso'] = iso
batters.loc[:,'babip'] = babip
batters.loc[:,'wrc'] = wrc
batters.loc[:,'wraa'] = wraa
batters.loc[:,'woba'] = woba
batters.loc[:,'wrc_plus'] = wrc_plus

<ipython-input-15-0499cefeeb18>:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  batter_df[col] = [float('NaN')]
/Users/nperry725/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/Users/nperry725/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1745: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] =

In [16]:
batters

,team code,game_date,game_number,mlb id,Name,batting order,confirmed,position,opp_pitch_hand,opp_pitch,...,bat_avg,obp,slg,ops,iso,babip,wrc,wraa,woba,wrc_plus
0,PIT,6/15/2021,1,624428,Adam Frazier,1,N,NaN,L,Patrick Corbin,...,0.263158,0.320388,0.400000,0.720388,0.136842,0.302632,12.389302,-0.048698,0.315698,98.911367
1,PIT,6/15/2021,1,663647,Ke'Bryan Hayes,2,N,NaN,L,Patrick Corbin,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,PIT,6/15/2021,1,668804,Bryan Reynolds,3,N,NaN,L,Patrick Corbin,...,0.293478,0.375000,0.510870,0.885870,0.217391,0.403226,17.904202,5.349609,0.378643,140.803722
3,PIT,6/15/2021,1,607732,Jacob Stallings,4,N,NaN,L,Patrick Corbin,...,0.265625,0.389610,0.546875,0.936485,0.281250,0.302326,14.800571,5.456544,0.400681,154.733712
4,PIT,6/15/2021,1,595943,Phillip Evans,5,N,NaN,L,Patrick Corbin,...,0.236842,0.309524,0.342105,0.651629,0.105263,0.275862,4.369694,-0.652318,0.290388,83.287519
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
278,LAD,6/15/2021,1,669257,Will Smith,4,N,NaN,R,Zach Eflin,...,0.308642,0.425000,0.604938,1.029938,0.296296,0.299213,43.343418,19.158044,0.431390,178.210642
279,LAD,6/15/2021,1,621035,Chris Taylor,5,N,NaN,R,Zach Eflin,...,0.283333,0.392226,0.450000,0.842226,0.166667,0.372671,47.437224,12.948708,0.370080,137.743822
280,LAD,6/15/2021,1,666158,Gavin Lux,6,N,NaN,R,Zach Eflin,...,0.234177,0.290698,0.398734,0.689432,0.164557,0.271930,18.268869,-2.303424,0.296797,90.348827
281,LAD,6/15/2021,1,656716,Zach McKinstry,7,N,NaN,R,Zach Eflin,...,0.270270,0.317073,0.594595,0.911668,0.324324,0.291667,7.095855,2.252981,0.380075,145.657095


In [17]:
k_9 = []
bb_9 = []
k_bb = []
hr_9 = []
k_pct = []
bb_pct = []
k_bb_pct = []
bat_avg = []
whip = []
babip = []
lob_pct = []
fip = []
xfip = []
for i, row in batters.iterrows():
    # Pulling stats based on handedness of pitcher
    pitcher_df = pitch_df.sort_values(by = 'IP', ascending = False).loc[(pitch_df['batter']== row['bats'])&(pitch_df['Name']==row['opp_pitch']),:]
    if len(pitcher_df) == 0:
        for col in pitcher_df.columns:
            pitcher_df[col] = [float('NaN')]
    k_9.append(pitcher_df['K/9'].values[0])
    bb_9.append(pitcher_df['BB/9'].values[0])
    k_bb.append(pitcher_df['K/BB'].values[0])
    hr_9.append(pitcher_df['HR/9'].values[0])
    k_pct.append(pitcher_df['K%'].values[0])
    bb_pct.append(pitcher_df['BB%'].values[0])
    k_bb_pct.append(pitcher_df['K-BB%'].values[0])
    bat_avg.append(pitcher_df['AVG'].values[0])
    whip.append(pitcher_df['WHIP'].values[0])
    babip.append(pitcher_df['BABIP'].values[0])
    lob_pct.append(pitcher_df['LOB%'].values[0])
    fip.append(pitcher_df['FIP'].values[0])
    xfip.append(pitcher_df['xFIP'].values[0])
batters.loc[:,'opp_k/9'] = k_9
batters.loc[:,'opp_bb/9'] = k_9
batters.loc[:,'opp_k/bb'] = k_9
batters.loc[:,'opp_hr/9'] = k_9
batters.loc[:,'opp_k_pct'] = k_9
batters.loc[:,'opp_bb_pct'] = k_9
batters.loc[:,'opp_k_bb_pct'] = k_9
batters.loc[:,'opp_avg'] = k_9
batters.loc[:,'opp_whip'] = k_9
batters.loc[:,'opp_babip'] = k_9
batters.loc[:,'opp_lob_pct'] = k_9
batters.loc[:,'opp_fip'] = k_9
batters.loc[:,'opp_xfip'] = k_9

/Users/nperry725/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/Users/nperry725/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1745: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)


In [18]:
pitchers = lineups.loc[(lineups[' batting order'] == 'SP')]

In [19]:
pitchers = pitchers[pitchers['Name'].isin(keep_pitch)]

In [20]:
batters

,team code,game_date,game_number,mlb id,Name,batting order,confirmed,position,opp_pitch_hand,opp_pitch,...,opp_hr/9,opp_k_pct,opp_bb_pct,opp_k_bb_pct,opp_avg,opp_whip,opp_babip,opp_lob_pct,opp_fip,opp_xfip
0,PIT,6/15/2021,1,624428,Adam Frazier,1,N,NaN,L,Patrick Corbin,...,9.108434,9.108434,9.108434,9.108434,9.108434,9.108434,9.108434,9.108434,9.108434,9.108434
1,PIT,6/15/2021,1,663647,Ke'Bryan Hayes,2,N,NaN,L,Patrick Corbin,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,PIT,6/15/2021,1,668804,Bryan Reynolds,3,N,NaN,L,Patrick Corbin,...,7.269231,7.269231,7.269231,7.269231,7.269231,7.269231,7.269231,7.269231,7.269231,7.269231
3,PIT,6/15/2021,1,607732,Jacob Stallings,4,N,NaN,L,Patrick Corbin,...,7.269231,7.269231,7.269231,7.269231,7.269231,7.269231,7.269231,7.269231,7.269231,7.269231
4,PIT,6/15/2021,1,595943,Phillip Evans,5,N,NaN,L,Patrick Corbin,...,7.269231,7.269231,7.269231,7.269231,7.269231,7.269231,7.269231,7.269231,7.269231,7.269231
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
278,LAD,6/15/2021,1,669257,Will Smith,4,N,NaN,R,Zach Eflin,...,9.126761,9.126761,9.126761,9.126761,9.126761,9.126761,9.126761,9.126761,9.126761,9.126761
279,LAD,6/15/2021,1,621035,Chris Taylor,5,N,NaN,R,Zach Eflin,...,9.126761,9.126761,9.126761,9.126761,9.126761,9.126761,9.126761,9.126761,9.126761,9.126761
280,LAD,6/15/2021,1,666158,Gavin Lux,6,N,NaN,R,Zach Eflin,...,10.607143,10.607143,10.607143,10.607143,10.607143,10.607143,10.607143,10.607143,10.607143,10.607143
281,LAD,6/15/2021,1,656716,Zach McKinstry,7,N,NaN,R,Zach Eflin,...,10.607143,10.607143,10.607143,10.607143,10.607143,10.607143,10.607143,10.607143,10.607143,10.607143


In [21]:
num_L_bats = []
num_R_bats = []
for i, row in pitchers.iterrows():
    opp_lineup = batters.loc[(batters['team code'] == matchups[row['team code']]),:]
    find_batters = []
    # There are batters whos data was null due to lack of plate appearances but handedness is known
    for j, row2 in opp_lineup.iterrows():
        if row2['bats'] != 'R' and row2['bats'] != 'L':
            find_bats = bat_df.loc[bat_df['Name'] == row2['Name'],:]
            if len(find_bats) > 0:
                find_batters.append((j, find_bats['bats'].values[0]))
            # If batter handedness is unknown, right is more likely
            if len(find_bats) == 0:
                find_batters.append((j, 'R'))
    for handedness in find_batters:
        opp_lineup.loc[handedness[0], 'bats'] = handedness[1]
    try:
        num_L_bats.append(opp_lineup['bats'].value_counts()['L'])
    except:
        num_L_bats.append(0)
    try:
        num_R_bats.append(opp_lineup['bats'].value_counts()['R'])
    except:
        num_R_bats.append(0)
pitchers['num_L_bats'] = num_L_bats
pitchers['num_R_bats'] = num_R_bats
is_nl = []
for i, row in pitchers.iterrows():
    if row['num_L_bats'] + row['num_R_bats'] == 9:
        is_nl.append(0)
    else:
        is_nl.append(1)
pitchers['is_nl'] = is_nl

/Users/nperry725/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


In [22]:
k_9_l = []
bb_9_l = []
k_bb_l = []
hr_9_l = []
k_pct_l = []
bb_pct_l = []
k_bb_pct_l = []
avg_l = []
whip_l = []
babip_l = []
lob_pct_l = []
fip_l = []
xfip_l = []
k_9_r = []
bb_9_r = []
k_bb_r = []
hr_9_r = []
k_pct_r = []
bb_pct_r = []
k_bb_pct_r = []
avg_r = []
whip_r = []
babip_r = []
lob_pct_r = []
fip_r = []
xfip_r = []
# Engineering features
k_9_l_tot = []
bb_9_l_tot = []
k_bb_l_tot = []
hr_9_l_tot = []
k_pct_l_tot = []
bb_pct_l_tot = []
k_bb_pct_l_tot = []
avg_l_tot = []
whip_l_tot = []
babip_l_tot = []
lob_pct_l_tot = []
fip_l_tot = []
xfip_l_tot = []
k_9_r_tot = []
bb_9_r_tot = []
k_bb_r_tot = []
hr_9_r_tot = []
k_pct_r_tot = []
bb_pct_r_tot = []
k_bb_pct_r_tot = []
avg_r_tot = []
whip_r_tot = []
babip_r_tot = []
lob_pct_r_tot = []
fip_r_tot = []
xfip_r_tot = []
for i, row in pitchers.iterrows():
    # Pulling stats of pitcher against lefty hitters
    pitcher_df_l = pitch_df.sort_values(by = 'IP', ascending = False).loc[(pitch_df['batter']== 'L')&(pitch_df['Name']==row['Name']),:]
    if len(pitcher_df_l) == 0:
        for col in pitcher_df_l.columns:
            pitcher_df_l[col] = [float('NaN')]
    k_9_l.append(pitcher_df_l['K/9'].values[0])
    bb_9_l.append(pitcher_df_l['BB/9'].values[0])
    k_bb_l.append(pitcher_df_l['K/BB'].values[0])
    hr_9_l.append(pitcher_df_l['HR/9'].values[0])
    k_pct_l.append(pitcher_df_l['K%'].values[0])
    bb_pct_l.append(pitcher_df_l['BB%'].values[0])
    k_bb_pct_l.append(pitcher_df_l['K-BB%'].values[0])
    avg_l.append(pitcher_df_l['AVG'].values[0])
    whip_l.append(pitcher_df_l['WHIP'].values[0])
    babip_l.append(pitcher_df_l['BABIP'].values[0])
    lob_pct_l.append(pitcher_df_l['LOB%'].values[0])
    fip_l.append(pitcher_df_l['FIP'].values[0])
    xfip_l.append(pitcher_df_l['xFIP'].values[0])
    # Feature Engineering
    k_9_l_tot.append(pitcher_df_l['K/9'].values[0] * row['num_L_bats'])
    bb_9_l_tot.append(pitcher_df_l['BB/9'].values[0] * row['num_L_bats'])
    k_bb_l_tot.append(pitcher_df_l['K/BB'].values[0] * row['num_L_bats'])
    hr_9_l_tot.append(pitcher_df_l['HR/9'].values[0] * row['num_L_bats'])
    k_pct_l_tot.append(pitcher_df_l['K%'].values[0] * row['num_L_bats'])
    bb_pct_l_tot.append(pitcher_df_l['BB%'].values[0] * row['num_L_bats'])
    k_bb_pct_l_tot.append(pitcher_df_l['K-BB%'].values[0] * row['num_L_bats'])
    avg_l_tot.append(pitcher_df_l['AVG'].values[0] * row['num_L_bats'])
    whip_l_tot.append(pitcher_df_l['WHIP'].values[0] * row['num_L_bats'])
    babip_l_tot.append(pitcher_df_l['BABIP'].values[0] * row['num_L_bats'])
    lob_pct_l_tot.append(pitcher_df_l['LOB%'].values[0] * row['num_L_bats'])
    fip_l_tot.append(pitcher_df_l['FIP'].values[0] * row['num_L_bats'])
    xfip_l_tot.append(pitcher_df_l['xFIP'].values[0] * row['num_L_bats'])
    # Pulling stats against righty hitters
    pitcher_df_r = pitch_df.sort_values(by = 'IP', ascending = False).loc[(pitch_df['batter']== 'R')&(pitch_df['Name']== row['Name']),:]
    if len(pitcher_df_r) == 0:
        for col in pitcher_df_r.columns:
            pitcher_df_r[col] = [float('NaN')]
    k_9_r.append(pitcher_df_r['K/9'].values[0])
    bb_9_r.append(pitcher_df_r['BB/9'].values[0])
    k_bb_r.append(pitcher_df_r['K/BB'].values[0])
    hr_9_r.append(pitcher_df_r['HR/9'].values[0])
    k_pct_r.append(pitcher_df_r['K%'].values[0])
    bb_pct_r.append(pitcher_df_r['BB%'].values[0])
    k_bb_pct_r.append(pitcher_df_r['K-BB%'].values[0])
    avg_r.append(pitcher_df_r['AVG'].values[0])
    whip_r.append(pitcher_df_r['WHIP'].values[0])
    babip_r.append(pitcher_df_r['BABIP'].values[0])
    lob_pct_r.append(pitcher_df_r['LOB%'].values[0])
    fip_r.append(pitcher_df_r['FIP'].values[0])
    xfip_r.append(pitcher_df_r['xFIP'].values[0])
    # Feature Engineering
    k_9_r_tot.append(pitcher_df_r['K/9'].values[0] * row['num_R_bats'])
    bb_9_r_tot.append(pitcher_df_r['BB/9'].values[0] * row['num_R_bats'])
    k_bb_r_tot.append(pitcher_df_r['K/BB'].values[0] * row['num_R_bats'])
    hr_9_r_tot.append(pitcher_df_r['HR/9'].values[0] * row['num_R_bats'])
    k_pct_r_tot.append(pitcher_df_r['K%'].values[0] * row['num_R_bats'])
    bb_pct_r_tot.append(pitcher_df_r['BB%'].values[0] * row['num_R_bats'])
    k_bb_pct_r_tot.append(pitcher_df_r['K-BB%'].values[0] * row['num_R_bats'])
    avg_r_tot.append(pitcher_df_r['AVG'].values[0] * row['num_R_bats'])
    whip_r_tot.append(pitcher_df_r['WHIP'].values[0] * row['num_R_bats'])
    babip_r_tot.append(pitcher_df_r['BABIP'].values[0] * row['num_R_bats'])
    lob_pct_r_tot.append(pitcher_df_r['LOB%'].values[0] * row['num_R_bats'])
    fip_r_tot.append(pitcher_df_r['FIP'].values[0] * row['num_R_bats'])
    xfip_r_tot.append(pitcher_df_r['xFIP'].values[0] * row['num_R_bats'])
pitchers['k_9_l'] = k_9_l
pitchers['bb_9_l'] = bb_9_l
pitchers['k_bb_l'] = k_bb_l
pitchers['hr_9_l'] = hr_9_l
pitchers['k_pct_l'] = k_pct_l
pitchers['bb_pct_l'] = bb_pct_l
pitchers['k_bb_pct_l'] = k_bb_pct_l
pitchers['bat_avg_l'] = avg_l
pitchers['whip_l'] = whip_l
pitchers['babip_l'] = babip_l
pitchers['lob_pct_l'] = lob_pct_l
pitchers['fip_l'] = fip_l
pitchers['xfip_l'] = xfip_l
# Right handed batter stats
pitchers['k_9_r'] = k_9_r
pitchers['bb_9_r'] = bb_9_r
pitchers['k_bb_r'] = k_bb_r
pitchers['hr_9_r'] = hr_9_r
pitchers['k_pct_r'] = k_pct_r
pitchers['bb_pct_r'] = bb_pct_r
pitchers['k_bb_pct_r'] = k_bb_pct_r
pitchers['bat_avg_r'] = avg_r
pitchers['whip_r'] = whip_r
pitchers['babip_r'] = babip_r
pitchers['lob_pct_r'] = lob_pct_r
pitchers['fip_r'] = fip_r
pitchers['xfip_r'] = xfip_r
# feature engineered left handed batter stats
pitchers['k_9_l_tot'] = k_9_l_tot
pitchers['bb_9_l_tot'] = bb_9_l_tot
pitchers['k_bb_l_tot'] = k_bb_l_tot
pitchers['hr_9_l_tot'] = hr_9_l_tot
pitchers['k_pct_l_tot'] = k_pct_l_tot
pitchers['bb_pct_l_tot'] = bb_pct_l_tot
pitchers['k_bb_pct_l_tot'] = k_bb_pct_l_tot
pitchers['bat_avg_l_tot'] = avg_l_tot
pitchers['whip_l_tot'] = whip_l_tot
pitchers['babip_l_tot'] = babip_l_tot
pitchers['lob_pct_l_tot'] = lob_pct_l_tot
pitchers['fip_l_tot'] = fip_l_tot
pitchers['xfip_l_tot'] = xfip_l_tot
# feature engineered righ handed batter stats
pitchers['k_9_r_tot'] = k_9_r_tot
pitchers['bb_9_r_tot'] = bb_9_r_tot
pitchers['k_bb_r_tot'] = k_bb_r_tot
pitchers['hr_9_r_tot'] = hr_9_r_tot
pitchers['k_pct_r_tot'] = k_pct_r_tot
pitchers['bb_pct_r_tot'] = bb_pct_r_tot
pitchers['k_bb_pct_r_tot'] = k_bb_pct_r_tot
pitchers['bat_avg_r_tot'] = avg_r_tot
pitchers['whip_r_tot'] = whip_r_tot
pitchers['babip_r_tot'] = babip_r_tot
pitchers['lob_pct_r_tot'] = lob_pct_r_tot
pitchers['fip_r_tot'] = fip_r_tot
pitchers['xfip_r_tot'] = xfip_r_tot

In [23]:
pitchers

,team code,game_date,game_number,mlb id,Name,batting order,confirmed,position,num_L_bats,num_R_bats,...,hr_9_r_tot,k_pct_r_tot,bb_pct_r_tot,k_bb_pct_r_tot,bat_avg_r_tot,whip_r_tot,babip_r_tot,lob_pct_r_tot,fip_r_tot,xfip_r_tot
8,PIT,6/15/2021,1,542881,Tyler Anderson,SP,N,NaN,2,6,...,7.280899,1.218997,0.538259,0.680739,1.495496,7.887640,1.670588,4.387352,26.192616,30.090019
17,WAS,6/15/2021,1,571578,Patrick Corbin,SP,N,NaN,2,6,...,11.328671,1.066975,0.457275,0.609700,1.908397,9.944056,2.093023,4.409091,32.275753,27.177962
27,NYY,6/15/2021,1,656756,Jordan Montgomery,SP,N,NaN,1,8,...,12.868085,1.831325,0.481928,1.349398,2.181818,10.621277,2.522523,5.402844,34.473577,32.208358
46,CHC,6/15/2021,1,621219,Alec Mills,SP,N,NaN,4,4,...,4.390244,0.830189,0.327044,0.503145,0.727273,3.804878,0.785047,2.909091,16.690390,16.531208
55,NYM,6/15/2021,1,592836,Taijuan Walker,SP,N,NaN,4,4,...,2.261780,0.880000,0.272000,0.608000,0.740088,3.706806,0.888889,3.049645,12.786471,16.345713
65,BAL,6/15/2021,1,518774,Matt Harvey,SP,N,NaN,4,5,...,8.181818,1.073620,0.368098,0.705521,1.733333,9.696970,2.090909,3.507194,22.398070,18.680228
75,CLE,6/15/2021,1,615698,Cal Quantrill,SP,N,NaN,4,5,...,6.428571,0.771812,0.369127,0.402685,1.423077,6.857143,1.509434,4.444444,24.966636,23.959359
84,BOS,6/15/2021,1,593958,Eduardo Rodriguez,SP,N,NaN,1,7,...,9.316901,1.801980,0.450495,1.351485,2.058824,10.056338,2.584615,5.171821,25.808124,22.934156
93,ATL,6/15/2021,1,656353,Tucker Davidson,SP,N,NaN,2,6,...,11.571429,0.571429,0.857143,-0.285714,1.371429,9.000000,1.200000,2.678571,41.991407,40.334653
103,TEX,6/15/2021,1,502043,Kyle Gibson,SP,N,NaN,3,6,...,5.108108,1.424051,0.569620,0.854430,1.462897,8.027027,1.850746,4.620390,22.237516,21.822174


**Data Description:** This worksheet provides a framework for creating dataframes for each days' players and stats. I will combine these dataframes across many days to get my final training data. These data frames have all of the stats that I could potentially be interested in, the features will be reduced for modeling as there is a significant amount of correlation between many of the features. The batters dataframe includes the batters stats against pitchers who throw with the same hand as the opposing pitcher, and includes the stats of how the opposing pitcher performs against batters with the same handedness of the batter. The pitchers dataframe contains each pitcher and their stats against both right handed and left handed batters. It also contains the number of right and left handed batters in the opposing lineup. I engineered features that are the pitchers stats vs the handedness of a batter multiplied by the number of those handed batters in the lineup. My final feature is is_nl. This is 1 if the lineup they are facing includes a pitcher and 0 if not.

**Data Dictionary:**   
team code: The team the player plays for.  
Name: The players name  
Batting order: The order the batters hit in the lineup. Is SP if player is a pitcher.  
opp_pitch_hand(batters): The hand that the opposing pitcher throws with  
opp_pitch(batters):The name of the opposing pitcher. 
opp_x(batters): This is the format for column names of stats that the opposing pitcher has against batters of the same handedness of the batter.  
(batters): There are also many columns of stats for how the batter performs against pitchers with the same handedness of the opposing pitcher.    
num_L_bats(pitchers): The number of left handed batters in the opposing lineup.  
num_R_bats(pitchers): The number of right handed batters in the opposing lineup.  
x_(l/r) (pitchers): stats describing how the pitcher performs against either left or right handed batters.    
x_(l/r)_tot (pitchers): The stats of how a pitcher performs against both hands multiplied by the number of those handed batters in the lineup.

In [24]:
scores = pd.read_csv('../datasets/6_15_target.csv', delimiter = ';')
scores.head()

,Date,GID,MLB_ID,Name,Starter,Bat order,DK posn,DK pts,DK sal,Team,Oppt,dblhdr,Tm Runs,Opp Runs
0,20210615,118v,592836,"Walker, Taijuan",1.0,9,1.0,36.15,8800.0,NYM,v chc,NaN,3,2
1,20210615,143d,623167,"Flexen, Chris",1.0,0,1.0,35.60,6100.0,SEA,v min,NaN,10,0
2,20210615,1057,474463,"Anderson, Brett",1.0,9,1.0,32.55,6900.0,MIL,v cin,NaN,1,2(F10)
3,20210615,111p,571578,"Corbin, Patrick",1.0,9,1.0,29.35,7000.0,WAS,v pit,NaN,8,1
4,20210615,130j,593423,"Montas, Frankie",1.0,0,1.0,28.15,8500.0,OAK,v laa,NaN,6,4


In [25]:
def clean_names(name):
    name = name.replace(',','')
    name = name.replace('.','')
    name = name.replace('-hitter','')
    name = name.replace('-pitcher','')
    names = name.split()
    first = names.pop(-1)
    names.insert(0, first)
    name = ' '.join(names)
    name = name.replace('James Happ', 'JA Happ')
    return name

In [26]:
scores['Name'] = [clean_names(name) for name in scores['Name']]

In [27]:
pitch_s = scores[scores['DK posn'] == 1]
hit_s = scores[scores['DK posn'] != 1]

In [28]:
pitch_final = pd.merge(left = pitchers, right = pitch_s, how = 'inner')

In [29]:
bat_final = pd.merge(left = batters, right = hit_s,
         how = 'inner').dropna(how = 'any', subset = ['bats', 'DK pts', 'Starter'])

In [30]:
bat_final = bat_final[bat_final['dblhdr'].isna()]

In [31]:
bat_final.to_csv('../merged_data/hitters_6_15.csv')
pitch_final.to_csv('../merged_data/pitchers_6_15.csv')

In [32]:
bat_final

,team code,game_date,game_number,mlb id,Name,batting order,confirmed,position,opp_pitch_hand,opp_pitch,...,Starter,Bat order,DK posn,DK pts,DK sal,Team,Oppt,dblhdr,Tm Runs,Opp Runs
2,PIT,6/15/2021,1,668804,Bryan Reynolds,3,N,NaN,L,Patrick Corbin,...,1.0,3,7.0,6.0,3200.0,PIT,@ was,NaN,1,8
3,PIT,6/15/2021,1,607732,Jacob Stallings,4,N,NaN,L,Patrick Corbin,...,1.0,4,2.0,0.0,3700.0,PIT,@ was,NaN,1,8
4,PIT,6/15/2021,1,595943,Phillip Evans,5,N,NaN,L,Patrick Corbin,...,1.0,8,37.0,7.0,2300.0,PIT,@ was,NaN,1,8
5,PIT,6/15/2021,1,570481,Erik Gonzalez,6,N,NaN,L,Patrick Corbin,...,1.0,6,35.0,6.0,2300.0,PIT,@ was,NaN,1,8
7,PIT,6/15/2021,1,621028,Kevin Newman,8,N,NaN,L,Patrick Corbin,...,1.0,1,6.0,0.0,2100.0,PIT,@ was,NaN,1,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
183,LAD,6/15/2021,1,457759,Justin Turner,2,N,NaN,R,Zach Eflin,...,1.0,3,5.0,3.0,5900.0,LAD,v phi,NaN,5,3
184,LAD,6/15/2021,1,669257,Will Smith,4,N,NaN,R,Zach Eflin,...,1.0,4,2.0,6.0,4900.0,LAD,v phi,NaN,5,3
185,LAD,6/15/2021,1,621035,Chris Taylor,5,N,NaN,R,Zach Eflin,...,1.0,6,4.0,6.0,5300.0,LAD,v phi,NaN,5,3
186,LAD,6/15/2021,1,666158,Gavin Lux,6,N,NaN,R,Zach Eflin,...,1.0,2,6.0,7.0,4300.0,LAD,v phi,NaN,5,3


In [33]:
pitch_final

,team code,game_date,game_number,mlb id,Name,batting order,confirmed,position,num_L_bats,num_R_bats,...,Starter,Bat order,DK posn,DK pts,DK sal,Team,Oppt,dblhdr,Tm Runs,Opp Runs
0,PIT,6/15/2021,1,542881,Tyler Anderson,SP,N,NaN,2,6,...,1.0,9,1.0,2.90,7900.0,PIT,@ was,NaN,1,8
1,WAS,6/15/2021,1,571578,Patrick Corbin,SP,N,NaN,2,6,...,1.0,9,1.0,29.35,7000.0,WAS,v pit,NaN,8,1
2,NYY,6/15/2021,1,656756,Jordan Montgomery,SP,N,NaN,1,8,...,1.0,0,1.0,6.60,7600.0,NYY,@ tor,NaN,6,5
3,CHC,6/15/2021,1,621219,Alec Mills,SP,N,NaN,4,4,...,1.0,9,1.0,11.55,6500.0,CHC,@ nym,NaN,2,3
4,NYM,6/15/2021,1,592836,Taijuan Walker,SP,N,NaN,4,4,...,1.0,9,1.0,36.15,8800.0,NYM,v chc,NaN,3,2
5,BAL,6/15/2021,1,518774,Matt Harvey,SP,N,NaN,4,5,...,1.0,0,1.0,-3.30,6000.0,BAL,@ cle,NaN,2,7
6,CLE,6/15/2021,1,615698,Cal Quantrill,SP,N,NaN,4,5,...,1.0,0,1.0,13.20,6800.0,CLE,v bal,NaN,7,2
7,BOS,6/15/2021,1,593958,Eduardo Rodriguez,SP,N,NaN,1,7,...,1.0,9,1.0,12.80,7800.0,BOS,@ atl,NaN,10,8
8,ATL,6/15/2021,1,656353,Tucker Davidson,SP,N,NaN,2,6,...,1.0,9,1.0,-0.35,7400.0,ATL,v bos,NaN,8,10
9,TEX,6/15/2021,1,502043,Kyle Gibson,SP,N,NaN,3,6,...,1.0,0,1.0,12.70,7500.0,TEX,@ hou,NaN,3,6(F10)
